In [3]:
import nltk
import random
import string 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [4]:

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/atwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/atwal/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /home/atwal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/atwal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
with open("panjab.txt", 'r', errors='ignore') as f:
    raw = f.read().lower()

sent_tokens= nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

print("Total sentences:", len(sent_tokens))
print("Total words:", len(word_tokens))


Total sentences: 54
Total words: 1466


In [7]:
from nltk.stem import WordNetLemmatizer

lemmer = nltk.stem.WordNetLemmatizer()
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


In [8]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
THANK_INPUTS = ("thanks", "thank you", "thank you so much")

def detect_intent(user_input):
    if user_input in THANK_INPUTS:
        return "thank_you"
    if user_input in GREETING_INPUTS:
        return "greeting"
    if "tell me more" in user_input:
        return "tell_me_more"
    if user_input == "bye":
        return "exit"
    return "faq" 


In [9]:
def faq_answer(user_input):
    global last_idx
    sent_tokens.append(user_input)

    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', token_pattern=None)
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]

    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    sent_tokens.remove(user_input)

    if req_tfidf == 0:
        last_idx = None
        return None
    else:
        last_idx = idx
        return sent_tokens[idx]

In [10]:
def tell_me_more():
    global last_idx
    if last_idx is not None and last_idx + 1 < len(sent_tokens):
        last_idx += 1
        return sent_tokens[last_idx]
    else:
        return None

In [13]:
def generate_response(intent, user_input):
    if intent == "greeting":
        return random.choice(["Hi!", "Hey!", "Hello! I’m glad you’re here."])
    elif intent == "thank_you":
        return "You're welcome!"
    elif intent == "tell_me_more":
        more_info = tell_me_more()
        return f"Sure! Here's more: {more_info}" if more_info else "I don’t have more details right now."
    elif intent == "faq":
        answer = faq_answer(user_input)
        return f"Here’s what I found: {answer}" if answer else "Hmm... I’m not sure. Could you ask differently?"
    elif intent == "exit":
        return "Goodbye! Take care."
    else:
        return "I didn’t quite get that."

In [14]:
# For frontend
def response(user_input):
    ui = user_input.lower().strip()
    intent = detect_intent(ui)
    return generate_response(intent, user_input)


In [ ]:
last_idx = None
if __name__ == "__main__":
    print("Chatbot: Hi! I am DeptBot 🤖. I can answer your queries about Department of Computer Science and Applications, Panjab University.\nType 'bye' to exit.")

    while True:
        user_response = input("You: ").lower().strip()
        intent = detect_intent(user_response)
        bot_reply = generate_response(intent, user_response)
        print("Chatbot:", bot_reply)
        if intent == "exit":
            break

Chatbot: Hi! I am DeptBot 🤖. I can answer your queries about Department of Computer Science and Applications, Panjab University.
Type 'bye' to exit.
Chatbot: Hey!
Chatbot: Here’s what I found: panjab university | brief history and present infrastructure
-------------------------------------------------------------
- one of the oldest universities in india, the panjab university (pu) initiated at lahore in 1882, has a long tradition of pursuing excellence in teaching and research in science and technology, humanities, social sciences, performing arts and sports.


/var/data/python/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'u', 'wa'] not in stop_words.
  warnings.warn(


Chatbot: Sure! Here's more: the university supports excellence and innovation in academic programmes, promotes excellence in research, scholarship and teaching.
Chatbot: Here’s what I found: department of computer science & applications (dcsa) | overview
---------------------------------------------------------------
- the computer culture at the panjab university dates back to 1966. an independent centre for computer science and applications (now a full fledged department) was set-up in 1983. the department aims at ingraining the spirit of ingenuity, innovativeness and technical competence in the students through rigorous competition and regular guidance.
Chatbot: Here’s what I found: panjab university | overview
-----------------------------
- established in 1882 in lahore and currently located in chandigarh.
Chatbot: Here’s what I found: recent faculty activities  
--------------------------
- tech utsav 2025 (inter-college tech fest) — april 4, 2025
- expert talk by dr. vijay kumar